In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Load data
df = pd.read_csv("/Users/mariahloehr/IICD/IICD/Data/cell_cycle_tidied.csv")

# Combine phase M and G2 into one class
df['phase'] = df['phase'].replace({'M': 'G2'})

# Separate features and target
X = df.drop(columns=['phase', 'age', 'PHATE_1', 'PHATE_2'])
y = df['phase']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=949, stratify=y)

In [3]:
# Define MLP model
mlp = MLPClassifier(max_iter=1000, random_state=949)

In [18]:
# Hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [
        (300,),         # 1 hidden layer
        (500,),        # 1 hidden layer
        (50,),        # 1 hidden layer
        (300, 150),     # 2 hidden layers
        (150, 75),     # 2 hidden layers
        (300, 150, 75), # 3 hidden layers
        (300,300,300)   # 3 hidden layers
    ]
}

# GridSearchCV
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Fit model
grid_search.fit(X_train, y_train)

# Output best parameters and best accuracy
print("Best parameters:", grid_search.best_params_)

Best parameters: {'hidden_layer_sizes': (300,)}


In [19]:
# Convert the cv_results_ dictionary to a DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Select and display relevant columns
print(
    results_df[
        [
            'param_hidden_layer_sizes',
            'mean_test_score',
            'std_test_score',
            'rank_test_score'
        ]
    ].sort_values(by='rank_test_score')
)

  param_hidden_layer_sizes  mean_test_score  std_test_score  rank_test_score
0                   (300,)         0.948164        0.007092                1
1                   (500,)         0.946469        0.006608                2
2                    (50,)         0.946469        0.003967                2
5           (300, 150, 75)         0.946469        0.006203                2
4                (150, 75)         0.946328        0.005053                5
6          (300, 300, 300)         0.946045        0.005607                6
3               (300, 150)         0.944209        0.007454                7


In [20]:
# Retrain with best parameters
best_mlp = grid_search.best_estimator_
best_mlp.fit(X_train, y_train)

# Predictions
y_train_pred = best_mlp.predict(X_train)
y_test_pred = best_mlp.predict(X_test)

In [21]:
# Evaluation
print("=== Training Set ===")
print("Overall Accuracy:", accuracy_score(y_train, y_train_pred))

print("\n=== Test Set ===")
print("Overall Accuracy:", accuracy_score(y_test, y_test_pred))

# Accuracy per phase (Train)
df_train = pd.DataFrame({'true': y_train, 'pred': y_train_pred})
accuracy_per_phase_train = df_train.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
print("\nAccuracy per phase (Train):")
print(accuracy_per_phase_train)

# Accuracy per phase (Test)
df_test = pd.DataFrame({'true': y_test, 'pred': y_test_pred})
accuracy_per_phase_test = df_test.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
print("\nAccuracy per phase (Test):")
print(accuracy_per_phase_test)

# Confusion Matrix
print("\nTest Confusion Matrix")
print(confusion_matrix(y_test, y_test_pred, labels=["G0", "G1", "G2", "S"]))

=== Training Set ===
Overall Accuracy: 1.0

=== Test Set ===
Overall Accuracy: 0.9423728813559322

Accuracy per phase (Train):
true
G0    1.0
G1    1.0
G2    1.0
S     1.0
dtype: float64

Accuracy per phase (Test):
true
G0    0.951743
G1    0.935331
G2    0.915966
S     0.956190
dtype: float64

Test Confusion Matrix
[[355  15   1   2]
 [ 15 593   5  21]
 [  1   2 218  17]
 [  1  13   9 502]]


/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_19841/1068295868.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_per_phase_train = df_train.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_19841/1068295868.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_per_phase_test = df_test.groupby('true').apply(

In [23]:
# save results
# === Load existing results ===
results_df = pd.read_csv("/Users/mariahloehr/IICD/IICD/Bar Plot/classification_results.csv", index_col=0)

# === Compute accuracy ===
from sklearn.metrics import accuracy_score

overall_acc = accuracy_score(y_test, y_test_pred)

df_test = pd.DataFrame({'true': y_test, 'pred': y_test_pred})
acc_per_phase = df_test.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))

# === Insert values ===
model_name = "MLP (full)"
results_df.loc[model_name, 'Overall'] = overall_acc

# Set per-phase accuracies
for phase in ['G0', 'G1', 'G2', 'S']:
    if phase in acc_per_phase.index:
        results_df.loc[model_name, phase] = acc_per_phase[phase]

# === Save updated file ===
results_df.to_csv("/Users/mariahloehr/IICD/IICD/Bar Plot/classification_results.csv")

/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_19841/2108438782.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acc_per_phase = df_test.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
